# Chapter 8: Object References, Mutability, and Recycling

TODO: Introduction here.

## Variables Are Not Boxes

Python variables are like reference variables in Java. From [Kode Java](https://kodejava.org/what-is-reference-variable-in-java/):

> A reference variable that is declared as final can’t never be reassigned to refer to a different object. The data within the object can be modified, but the reference variable cannot be changed.

With reference variables, it makes much more sense to say that the variable is assigned to an object, and not the other way around.

In [2]:
a = [1, 2, 3]
b = a
a.append(4)
b

[1, 2, 3, 4]

In the above example, `a` and `b` hold references to *the same list*. A helpful analogy is that we can think of variables as sticky notes that we label an object with, instead of boxes that hold objects.

In [4]:
# Example 8-2. Variables are assigned to objects only after the objects are created

class Gizmo():
    def __init__(self):
        print(f"Gizmo id: {id(self)}")
        
x = Gizmo()
y = Gizmo() * 10

Gizmo id: 4429747248
Gizmo id: 4429748592


TypeError: unsupported operand type(s) for *: 'Gizmo' and 'int'

Multiplying the `Gizmo` instance will raise an exception, but is proof that a second `Gizmo` was actually instantiated *before* the multiplication was attempted.

But the variable `y` was never created, because the exceptino happened while the right-hand side of the assignment was being evaluated.

> To understand an assignment in Python, *always read the right-hand side first*: that's where the object is created or retrieved. After that, the variable on the left is bound to the object, like a label stuck to it.

## Identity, Equality and Aliases

Because variable are mere labels, nothing prevents an object from having several labels assigned to it.

The following chunk is an example of *aliasing*.

In [9]:
# Example 8-3. `charles` and `lewis` refer to the same object

charles = {"name": "Charles L. Dodgson", "born": 1832}
lewis = charles

print(lewis is charles)
print(id(charles), id(lewis))

True
4429902144 4429902144


In [10]:
lewis["balance"] = 950
charles

{'name': 'Charles L. Dodgson', 'born': 1832, 'balance': 950}

In [11]:
# Example 8-4. `alex` and `charles` compare equal, but `alex` is not `charles`
alex = {"name": "Charles L. Dodgson", "born": 1832, "balance": 950}

print(alex == charles)
print(alex is charles)

True
False


In example 8-3, `charles` and `lewis` are aliases: **two variables bound to the same object**.

In example 8-4, `alex` is not an alias for `charles`: **these variables are bound to distinc objects**.

The objects bound to `alex` and `charles` have the same *value*, but they have different identities.

They compare equal because of the `__eq__` implementation in the `dict class`, but are distinct objects.

The Pythonic way of writing negative identity comparison is `a is not b`.

## Choosing between `==` and `is`

The `==` operator compares the values of objects (i.e. the data they hold), while `is` compares identities.

We often care more about values in practice that identities, that is why `==` is so prominent in Python code.

However, when comparing a variable to a [singleton](https://en.wikipedia.org/wiki/Singleton_variable), then it makes sense to use `is`.

> In computer programming a singleton variable is a variable that is referred to only once. Examples of where a variable might only be referenced once is as a dummy argument in a function call, or when its address is assigned to another variable which subsequently accesses its allocated storage.

The most common case is checking whether a variable is bound to `None`.

The `is` operator is faster than `==`, because it cannot be overloaded.

In [16]:
a = None
b = None
print(id(a), id(b))

4390919400 4390919400


The `==` operator is syntactic sugar for `a.__eq__(b)`.

The `__eq__` method inherited from `object` compares object IDs and produces the same results as `is`.

However, **most built-in types override `__eq__`** with more meaningful implementations that take into account the values of the object attributes.

## The Relative Immutability of Tuples

Tuples hold references to objects in Python (in contrast to single-type sequences described in chapter XX which physically hold their data in contiguous memory).

The immutability of tuples refers to the physical contents of the `tuple` data structure (i.e. the references it holds) and does not extend to the referenced objects.

In [18]:
# Example 8-5. `t1` and `t2` intially compare equal, but chaning a mutable item inside tuple `t1` makes i different

# `t1` is immutable, but t1[-1] is mutable
t1 = (1, 2, [30, 40])
t2 = (1, 2, [30, 40])

print(t1 == t2)

print(id(t1[-1]))
print(id(t2[-1]))

t1[-1].append(99)
print(t1[-1])

print(t1 == t2)

True
4429186944
4429187520
[30, 40, 99]
False


The relative immutability of tuples is the reason why some tuples are unhashable.

The distinction between equality and identity has implications when you need to copy an object.

## Copies Are Shallow by Default

A copy is an equal object with a different ID. But if an object contains other objects, should the copy also duplicate their inner objects, or is it OK to share them?

The easiest way to copy most built-in mutable collections is to use the built-in constructor for the type itself.

In [21]:
l1 = [3, [55, 44], [7, 8, 9]]
l2 = list(l1)
l2

[3, [55, 44], [7, 8, 9]]

In [22]:
print(l2 == l1)
print(l2 is l1)

True
False


For lists, the shortcut `l2 = l1[:]` also makes a copy.

However, using the constructor or the shortcut produces a **shallow copy** filled with references to the same items held in by the original container.

This saves memory and causes no problems if the items are immutable, but if there are mutable items it may lead to unpleasant surprises.

In [24]:
# Example 8-6. Making a shallow copy of a list containing another list

l1 = [2, [66, 55, 44], (7, 8, 9)]
l2 = list(l1)
l1.append(100)
l1[1].remove(55)

print("l1:", l1)
print("l2:", l2)

l2[1] += [33, 22]
l2[2] += (10, 11)

print("l1:", l1)
print("l2:", l2)

l1: [2, [66, 44], (7, 8, 9), 100]
l2: [2, [66, 44], (7, 8, 9)]
l1: [2, [66, 44, 33, 22], (7, 8, 9), 100]
l2: [2, [66, 44, 33, 22], (7, 8, 9, 10, 11)]


Here is what is going on, step-by-step:

1. `l2` is a shallow copy of `l1`.

2. Appending `100` to `l1` has no effect on `l2`.

3. Removing `55` from the inner list in `l1[1]` affects `l2` because `l2[1]` is bound to the same list.

4. For a mutable object like the list referred to by `l2[1]`, the `+=` operator changes the list in place. This affects `l1[1]`, which is an alias for `l2[1]`.

5. `+=` on a tuple creates a new tuple and rebinds the variable `l2[2]` here. Now the tuples in the last position are no longer references to the same objects.

Ramalho recommends copying and pasting the above chunk and into the [Online Python Tutor](http://pythontutor.com/visualize.html#mode=edit) to visualise what's going on. The following figure shows the final state of the program:

<img src="../figs/shallow-copies.png" alt="Figure" style="width: 500px;"/>

## Deep and Shallow Objects of Arbitrary Objects

The `copy` module provides the `deepcopy` and `copy` functions that return deep and shallow copies of arbitrary objects.

In [25]:
# Example 8-8. Bus picks up and drops off passengers

class Bus:
    
    def __init__(self, passengers=None):
        if passengers is None:
            self.passengers = []
        else:
            self.passengers = list(passengers)
            
    def pick(self, name):
        self.passengers.append(name)
        
    def drop(self, name):
        self.passengers.remove(name)
    

In [28]:
import copy

bus1 = Bus(["Alice", "Bill", "Claire", "David"])
bus2 = copy.copy(bus1)
bus3 = copy.deepcopy(bus1)

id(bus1), id(bus2), id(bus3)

(4430050544, 4429749696, 4429749456)

In [29]:
bus1.drop("Bill")
bus2.passengers # Now missing Bill ...

['Alice', 'Claire', 'David']

In [30]:
id(bus1.passengers), id(bus2.passengers), id(bus3.passengers) # ... because bus1 and bus2 reference same list.

(4430657472, 4430657472, 4430322560)

In [31]:
bus3.passengers # Not missing Bill

['Alice', 'Bill', 'Claire', 'David']

The `deepcopy` function remembers objects already copied to hande cyclic references (thereby avoiding infinite loops), demonstrated in the following example:

In [32]:
a = [10, 20]
b = [a, 30]
a.append(b)
a

[10, 20, [[...], 30]]

In [34]:
c = copy.deepcopy(a)
c

[10, 20, [[...], 30]]

In some cases a deep copy may be too deep. For example,objects may refer to external resources or singletons that should not be copied.

The behaviour of both `copy` and `deepcopy` can be controlled by modifiying the `__copy__()` and `__deepcopy__()` special methods.

## Function Parameters as References

